In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score, f1_score
from sklearn.impute import SimpleImputer, KNNImputer
import numpy as np

# chargement jeu de donnée avec le moins de features
df = pd.read_csv('movies_clean_final.csv')
df.head(20)



,acteurs,budget,compositeur,date,duree,entrees_premiere_semaine,franchise,genre,pays,producteur,realisateur,remake,salles_premiere_semaine,studio,titre,scoring_acteurs,scoring_acteurs_realisateurs,season,coeff_studio
0,"Dany Boon, Kad Merad, Michel Galabru, Jérôme C...",12547392,NaN,2008-02-27,106,4378720,NaN,Comédie,France,Claude Berri,Dany Boon,Remake,793,Pathé,Bienvenue chez les Ch'tis,0.275862,0.482759,Hiver,2
1,"Christian Clavier, Jamel Debbouze, Gérard Depa...",50300000,NaN,2002-01-30,107,3685097,Franchise,Comédie,France,Claude Berri,Alain Chabat,NaN,945,Pathé,Astérix et Obélix: Mission Cléopatre,1.310345,1.344828,Hiver,2
2,"Ewan McGregor, Natalie Portman, Samuel L. Jackson",113000000,John Williams,2005-05-18,146,3303005,Franchise,Fantasy,Etats-Unis,George Lucas,George Lucas,NaN,938,20th Century Fox,La Revanche des Sith,0.448276,0.448276,Printemps,3
3,"Chris Evans, Chris Hemsworth, Robert Downey Jr...",356000000,Alan Silvestri,2019-04-24,181,3426471,Franchise,Comicbook,Etats-Unis,Kevin Feige,Russo (brothers),NaN,633,Walt Disney Pictures,Avengers: Endgame,0.931034,0.931034,Printemps,3
4,"Sam Worthington, Sigourney Weaver, Kate Winslet",350000000,NaN,2022-12-14,192,2739848,Franchise,Science Fiction,Etats-Unis,NaN,James Cameron,NaN,762,Walt Disney Pictures,Avatar : la voie de l'eau,0.448276,0.724138,Hiver,3
5,"Benedict Cumberbatch, Tom Holland, Willem Dafo...",200000000,NaN,2021-12-15,148,2867515,Franchise,Comicbook,Etats-Unis,Kevin Feige,Jon Watts,NaN,882,Sony Pictures,Spider-Man: No Way Home,0.000000,0.000000,Hiver,3
6,"Daniel Radcliffe, Emma Watson, Helena Bonham C...",200000000,NaN,2009-07-15,153,2882397,Franchise,Fantasy,Etats-Unis,NaN,David Yates,NaN,949,Warner Bros.,Harry Potter et le Prince de sang-mêlé,0.482759,0.482759,Été,3
7,"Orlando Bloom, Viggo Mortensen, Ian McKellen, ...",94000000,Howard Shore,2003-12-17,201,2852784,Franchise,Fantasy,Etats-Unis,NaN,Peter Jackson,NaN,993,Metropolitan,Le Seigneur des anneaux: Le Retour du roi,1.034483,1.034483,Hiver,0
8,"Richard Anconina, José Garcia, Gad Elmaleh",12560000,NaN,2001-02-07,105,2830489,Franchise,Comédie,France,NaN,Thomas Gilou,NaN,827,Warner Bros.,La Vérité si je mens! 2,0.034483,0.034483,Hiver,3
9,"Gérard Depardieu, Christian Clavier, Roberto B...",41900000,NaN,1999-02-03,109,2718443,Franchise,Comédie,France,Claude Berri,Claude Zidi,NaN,780,Pathé,Astérix et Obélix contre César,0.517241,0.517241,Hiver,2


In [4]:
df.shape

(4549, 19)

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_absolute_error, root_mean_squared_error, r2_score
import pandas as pd
df = pd.read_csv('movies_clean_final2.csv')

df = df.drop(['acteurs', 'scoring_acteurs', 'realisateur', 'titre'], axis=1)

# Gestion des valeurs manquantes pour les colonnes numériques et catégorielles
numeric_features = df.select_dtypes(include=['int64', 'float64']).columns.drop('entrees_premiere_semaine')
numeric_transformer = SimpleImputer(strategy='median')

categorical_features = df.select_dtypes(include=['object']).columns
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Préparation du préprocesseur
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    #     ('poly', PolynomialFeatures(degree=2, include_bias=False), numeric_features)
    ])

# Modèle de régression
model = Pipeline(steps=[('preprocessor', preprocessor),
                        ('regressor', RandomForestRegressor(n_estimators=100, random_state=0))])

# Séparation des données
X = df.drop('entrees_premiere_semaine', axis=1)
y = df['entrees_premiere_semaine']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Entraînement du modèle
model.fit(X_train, y_train)

# Prédiction sur l'ensemble de test
y_pred = model.predict(X_test)

# Évaluation du modèle
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
rmse = root_mean_squared_error(y_test, y_pred)
print(mae)
print(r2)
print(rmse)

128319.3575889328
0.7519012677070012
247292.37257577275


In [8]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_absolute_error, root_mean_squared_error, r2_score
import pandas as pd
df = pd.read_csv('movies_clean_final2.csv')

df = df.drop(['acteurs', 'scoring_acteurs'], axis=1)

# Gestion des valeurs manquantes pour les colonnes numériques et catégorielles
numeric_features = df.select_dtypes(include=['int64', 'float64']).columns.drop('entrees_premiere_semaine')
numeric_transformer = SimpleImputer(strategy='median')

categorical_features = df.select_dtypes(include=['object']).columns
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Préparation du préprocesseur
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features),
        ('poly', PolynomialFeatures(degree=1, include_bias=False), numeric_features)
    ])

# Modèle de régression
model = Pipeline(steps=[('preprocessor', preprocessor),
                        ('regressor', RandomForestRegressor(n_estimators=100, random_state=0))])

# Séparation des données
X = df.drop('entrees_premiere_semaine', axis=1)
y = df['entrees_premiere_semaine']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Entraînement du modèle
model.fit(X_train, y_train)

# Prédiction sur l'ensemble de test
y_pred = model.predict(X_test)

# Évaluation du modèle
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
rmse = root_mean_squared_error(y_test, y_pred)
print(mae)
print(r2)
print(rmse)

137866.23225296443
0.7232919157160633
261161.6312097228


In [ ]:
new_films = pd.DataFrame({
    'titre': ['NOUS, LES LEROY'],  # Assurez-vous que les valeurs soient dans des listes
    'budget': [None],
    'genre': ['Comédie'],
    'pays': ['France'],
    #'acteurs': [['George Clooney', 'CA', 'NJ']],  # Cela dépend de comment vous avez traité cette caractéristique
    'producteur': ['missing'],
    'realisateur': ['missing'],
    'compositeur': ['missing'],
    'studio': ['Apollo Films'],
    'coeff_studio': [1],
    'date': ['2024-04-10'],
    'season': ['Hiver'],
    'scoring_acteurs&realisateur': [0.5],
    'remake': [None],
    'franchise': [None],
})
# faire des prédictions avec le modèle optimisé
new_y_pred = model.predict(new_films)

# ajouter les prédictions à new_data
new_films['pred_entrees'] = new_y_pred
new_films['pred_entrees_dans_cine_jour'] = (new_y_pred/2000)/7
print(new_films)

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor

from sklearn.metrics import mean_absolute_error, r2_score, root_mean_squared_error
import pandas as pd
df = pd.read_csv('movies_clean_final2.csv')

df = df.drop(['acteurs', 'scoring_acteurs', 'realisateur', 'studio'], axis=1)

# Gestion des valeurs manquantes pour les colonnes numériques et catégorielles
numeric_features = df.select_dtypes(include=['int64', 'float64']).columns.drop('entrees_premiere_semaine')
numeric_transformer = SimpleImputer(strategy='median')

categorical_features = df.select_dtypes(include=['object']).columns
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Préparation du préprocesseur
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features),
        # ('poly', PolynomialFeatures(degree=1, include_bias=False), numeric_features)  # Ajouter cette ligne pour les fonctionnalités polynomiales
    ])

# Modèle de régression
model = Pipeline(steps=[('preprocessor', preprocessor),
                        ('regressor', CatBoostRegressor())])

# Séparation des données
X = df.drop('entrees_premiere_semaine', axis=1)
y = df['entrees_premiere_semaine']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Entraînement du modèle
model.fit(X_train, y_train)

# Prédiction sur l'ensemble de test
y_pred = model.predict(X_test)

# Évaluation du modèle
mae = mean_absolute_error(y_test, y_pred)
rmse = root_mean_squared_error(y_test, y_pred)
mae
r2 = r2_score(y_test, y_pred)
print(r2)
print(rmse)



Learning rate set to 0.048788
0:	learn: 416482.1028476	total: 63.1ms	remaining: 1m 3s
1:	learn: 404944.1205715	total: 74.4ms	remaining: 37.1s
2:	learn: 393458.6344834	total: 85.6ms	remaining: 28.5s
3:	learn: 383354.2785887	total: 96.7ms	remaining: 24.1s
4:	learn: 373625.7247013	total: 107ms	remaining: 21.3s
5:	learn: 364799.2958064	total: 118ms	remaining: 19.5s
6:	learn: 356056.6909422	total: 127ms	remaining: 18s
7:	learn: 348079.5770804	total: 135ms	remaining: 16.7s
8:	learn: 340785.8814057	total: 142ms	remaining: 15.6s
9:	learn: 334314.5880784	total: 150ms	remaining: 14.8s
10:	learn: 327310.1794592	total: 158ms	remaining: 14.2s
11:	learn: 321214.7227200	total: 166ms	remaining: 13.7s
12:	learn: 315762.6016698	total: 175ms	remaining: 13.3s
13:	learn: 310337.5661347	total: 183ms	remaining: 12.9s
14:	learn: 305162.6500415	total: 191ms	remaining: 12.6s
15:	learn: 300669.8829660	total: 199ms	remaining: 12.2s
16:	learn: 296129.8763633	total: 206ms	remaining: 11.9s
17:	learn: 291770.1612732	

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import xgboost
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import mean_absolute_error
import pandas as pd
df = pd.read_csv('movies_clean_final.csv')

df = df.drop(['acteurs', 'scoring_acteurs', 'realisateur', 'studio'], axis=1)

# Gestion des valeurs manquantes pour les colonnes numériques et catégorielles
numeric_features = df.select_dtypes(include=['int64', 'float64']).columns.drop('entrees_premiere_semaine')
numeric_transformer = SimpleImputer(strategy='median')

categorical_features = df.select_dtypes(include=['object']).columns
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Préparation du préprocesseur
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Modèle de régression
model = Pipeline(steps=[('preprocessor', preprocessor),
                        ('regressor', xgboost.XGBRegressor())])

# Séparation des données
X = df.drop('entrees_premiere_semaine', axis=1)
y = df['entrees_premiere_semaine']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

parameters = {'regressor__nthread':[4], #when use hyperthread, xgboost may become slower
              'regressor__objective':['reg:linear'],
              'regressor__learning_rate': [.03, 0.05, .07], #so called `eta` value
              'regressor__max_depth': [5, 6, 7],
              'regressor__min_child_weight': [4],
              'regressor__silent': [1],
              'regressor__subsample': [0.7],
              'regressor__colsample_bytree': [0.7],
              'regressor__n_estimators': [500]}


# Créer une instance de GridSearchCV
grid_search = GridSearchCV(model, parameters, cv=5, n_jobs=5, verbose=True)

# Effectuer la recherche sur la grille
grid_search.fit(X_train, y_train)

# Meilleurs hyperparamètres trouvés
best_params = grid_search.best_params_

# Meilleur score obtenu sur l'ensemble d'entraînement
best_score = grid_search.best_score_

# Meilleur modèle
best_model = grid_search.best_estimator_

# Utiliser le meilleur modèle pour prédire les étiquettes sur l'ensemble de test
y_pred = best_model.predict(X_test)

print("Meilleurs hyperparamètres:", best_params)
print("Meilleur score sur l'ensemble d'entraînement:", best_score)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


/home/utilisateur/Documents/cinema_prediction/.venv/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [10:24:19] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)
/home/utilisateur/Documents/cinema_prediction/.venv/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [10:24:19] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)
/home/utilisateur/Documents/cinema_prediction/.venv/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [10:24:19] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)
/home/utilisateur/Documents/cinema_prediction/.venv/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [10:24:19] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "silent" } are not used

Meilleurs hyperparamètres: {'regressor__colsample_bytree': 0.7, 'regressor__learning_rate': 0.03, 'regressor__max_depth': 5, 'regressor__min_child_weight': 4, 'regressor__n_estimators': 500, 'regressor__nthread': 4, 'regressor__objective': 'reg:linear', 'regressor__silent': 1, 'regressor__subsample': 0.7}
Meilleur score sur l'ensemble d'entraînement: 0.520818525886079


In [17]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
from sklearn.linear_model import Lasso

from sklearn.metrics import mean_absolute_error, r2_score, root_mean_squared_error
import pandas as pd
df = pd.read_csv('movies_clean_final2.csv')

df = df.drop(['acteurs', 'scoring_acteurs', 'realisateur', 'studio'], axis=1)

# Gestion des valeurs manquantes pour les colonnes numériques et catégorielles
numeric_features = df.select_dtypes(include=['int64', 'float64']).columns.drop('entrees_premiere_semaine')
numeric_transformer = SimpleImputer(strategy='median')

categorical_features = df.select_dtypes(include=['object']).columns
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Préparation du préprocesseur
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features),
        ('poly', PolynomialFeatures(degree=2, include_bias=False), numeric_features)  # Ajouter cette ligne pour les fonctionnalités polynomiales
    ])

# Modèle de régression
model = Pipeline(steps=[('preprocessor', preprocessor),
                        ('regressor', Lasso(alpha=0.1))])

# Séparation des données
X = df.drop('entrees_premiere_semaine', axis=1)
y = df['entrees_premiere_semaine']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Entraînement du modèle
model.fit(X_train, y_train)

# Prédiction sur l'ensemble de test
y_pred = model.predict(X_test)

# Évaluation du modèle
mae = mean_absolute_error(y_test, y_pred)
rmse = root_mean_squared_error(y_test, y_pred)
mae
r2 = r2_score(y_test, y_pred)
print(r2)
print(rmse)

0.5823497278784961
320852.0608012206


/home/utilisateur/Documents/cinema_prediction/.venv/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:639: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 136931795883.28342, tolerance: 55619245151.67537
  model = cd_fast.sparse_enet_coordinate_descent(


In [9]:
from pycaret.datasets import get_data
import pandas as pd 
df = df = pd.read_csv('movies_clean_final.csv')

df = df.drop(['acteurs', 'scoring_acteurs', 'realisateur', 'studio'], axis=1)

df.to_csv('df_pycaret.csv')

dataset = get_data('df_pycaret')
#check the shape of data
dataset.shape


ModuleNotFoundError: No module named 'pycaret'

In [8]:
from pycaret.regression import *

s = setup(data = dataset, target = 'entrees_premiere_semaine', session_id=123)

,Description,Value
0,Session id,123
1,Target,entrees_premiere_semaine
2,Target type,Regression
3,Original data shape,"(4541, 14)"
4,Transformed data shape,"(4541, 50)"
5,Transformed train set shape,"(3178, 50)"
6,Transformed test set shape,"(1363, 50)"
7,Numeric features,4
8,Categorical features,9
9,Rows with missing values,100.0%


In [11]:
best = compare_models()
best

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,12:43:10
Status,. . . . . . . . . . . . . . . . . .,Loading Dependencies
Estimator,. . . . . . . . . . . . . . . . . .,Compiling Library


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,234461.8800,149732496938.5104,384676.3356,0.1845,1.6120,17.2992,0.4460
knn,K Neighbors Regressor,224809.2574,150346439580.0738,386293.4335,0.1742,1.5050,13.4622,0.0870
catboost,CatBoost Regressor,238151.2590,157846148987.2522,395220.5592,0.1391,1.6170,17.4247,1.3980
gbr,Gradient Boosting Regressor,270184.7894,183072710177.3636,426076.4254,-0.0024,1.6968,19.0106,0.2420
rf,Random Forest Regressor,270091.1438,183169132272.3973,426176.5397,-0.0028,1.6968,18.9933,0.4680
dt,Decision Tree Regressor,270149.9334,183192797114.6479,426212.0873,-0.0030,1.6970,19.0013,0.0890
ridge,Ridge Regression,270485.2140,183412852633.9713,426466.0006,-0.0041,1.6977,19.0116,0.0780
lr,Linear Regression,270485.2140,183412852634.2998,426466.0006,-0.0041,1.6977,19.0116,0.3920
lasso,Lasso Regression,270484.9638,183412527193.8150,426465.6337,-0.0041,1.6977,19.0116,0.0800
en,Elastic Net,270485.1122,183412856345.3961,426465.9931,-0.0041,1.6977,19.0116,0.0780


ExtraTreesRegressor(n_jobs=-1, random_state=123)